# **Resolución TP1: Algoritmos de búsqueda en Torre de Hanoi**

##1. ¿Cuáles son los PEAS de este problema?

### - **Perfomance**: partiendo del estado inicial (5 discos apilados de en varilla 1) mover
